<a href="https://colab.research.google.com/github/Heavymetal17/inter-hall-data-analytics/blob/main/XGB_LGBM_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
from datetime import datetime
import pandas as pd
from xgboost import XGBRegressor

matplotlib.rcParams['figure.figsize'] = (10, 8)
matplotlib.rcParams['axes.grid'] = False

In [ ]:
df = pd.read_excel('/content/Train_data.xlsx')
df = df.melt(id_vars=['Warehouse id', 'Region', 'SKU id'], var_name='date', value_name='sales')
df['SKU id'] = df['SKU id'].str.extract('(\d+)',expand = False).astype(int)
df.rename(columns={'SKU id':'store_id'}, inplace=True)

In [ ]:
df['sales previous month'] = df.groupby(['store_id', 'Region'])['sales'].shift(1)
df['sales next month'] = df.groupby(['store_id', 'Region'])['sales'].shift(-1)
df['diff sales 1'] = df.groupby(['store_id', 'Region'])['sales'].diff(1)
df.sort_values(['store_id', 'Region', 'date'], inplace=True)
df['rolling mean sales 4'] = df.groupby('store_id')['sales'].rolling(4).mean().reset_index(level=0, drop=True)

df['isFestivalMth'] = df['date'].apply(lambda x: x.month == 1 or x.month == 3 or x.month == 10 or x.month == 12).astype(int)
df['isSummer'] = df['date'].apply(lambda x: x.month == 4 or x.month == 5 or x.month == 6).astype(int)
df['isWinter'] = df['date'].apply(lambda x: x.month == 11 or x.month == 12 or x.month == 1 or x.month == 2).astype(int)

df['Year'] = df['date'].apply(lambda time: time.year)
df['Month'] = df['date'].apply(lambda time: time.month)

df.reset_index(level=0, drop=True, inplace=True)
# df.drop('date', axis=1, inplace=True)
df.fillna(df.median(), inplace=True)
df = df[['Year', 'Month', 'Warehouse id', 'store_id', 'sales', 'sales previous month', 'sales next month', 'diff sales 1', 'rolling mean sales 4', 'isFestivalMth', 'isSummer', 'isWinter', 'Region', 'date']]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.


In [ ]:
split_point = '2021-01-01'
df_test_copy = df[df['date'] >= split_point].copy().reset_index(drop=True)

In [ ]:
warehouse_encode = pd.get_dummies(df['Warehouse id'], prefix = 'Warehouse')
region_encode = pd.get_dummies(df['Region'], prefix = 'Region')
df.drop(['Warehouse id', 'Region'], axis =1, inplace = True)
df = pd.concat([warehouse_encode, region_encode, df], axis = 1)
df.head() 

,Warehouse_Wh-1,Warehouse_Wh-2,Warehouse_Wh-3,Warehouse_Wh-4,Region_EAST,Region_NORTH,Region_SOUTH,Region_WEST,Year,Month,...,store_id,sales,sales previous month,sales next month,diff sales 1,rolling mean sales 4,isFestivalMth,isSummer,isWinter,date
0,0,1,0,0,1,0,0,0,2018,4,...,1,23,36.0,20.0,0.0,53.00,0,1,0,2018-04-01
1,0,1,0,0,1,0,0,0,2018,5,...,1,20,23.0,11.0,-3.0,53.00,0,1,0,2018-05-01
2,0,1,0,0,1,0,0,0,2018,6,...,1,11,20.0,23.0,-9.0,53.00,0,1,0,2018-06-01
3,0,1,0,0,1,0,0,0,2018,7,...,1,23,11.0,53.0,12.0,19.25,0,0,0,2018-07-01
4,0,1,0,0,1,0,0,0,2018,8,...,1,53,23.0,5.0,30.0,26.75,0,0,0,2018-08-01


In [ ]:
df_N = df[df['Region_NORTH'] == 1].reset_index(drop=True)
df_S = df[df['Region_SOUTH'] == 1].reset_index(drop=True)
df_E = df[df['Region_EAST'] == 1].reset_index(drop=True)
df_W = df[df['Region_WEST'] == 1].reset_index(drop=True)
df_NE = pd.concat([df_N, df_E]).reset_index(drop=True)
df_NW = pd.concat([df_N, df_W]).reset_index(drop=True)
df_SE = pd.concat([df_S, df_E]).reset_index(drop=True)
df_SW = pd.concat([df_S, df_W]).reset_index(drop=True)

In [ ]:
def mape(y_pred, y_train):
    ape = np.abs((y_pred - y_train) / y_train)
    ape[~np.isfinite(ape)] = 1
    return np.mean(ape)

In [ ]:
split_point = '2021-01-01'

In [ ]:
# df_dict = {'All': df, 'N': df_N, 'S': df_S, 'E': df_E, 'W': df_W, 'NE': df_NE, 'NW': df_NW, 'SE': df_SE, 'SW': df_SW}
# for key, value in df_dict.items():
#     df = df_dict[key]
    
#     df_test = df[df['date'] >= split_point].copy()
#     df_test.drop('date', axis=1, inplace=True)
#     df_test.reset_index(drop=True, inplace=True)
#     df_train = df[df['date'] < split_point].copy()
#     df_train.drop('date', axis=1, inplace=True)
#     df_train.reset_index(drop=True, inplace=True)

#     model = XGBRegressor(objective='reg:squarederror', n_estimators=1000)

#     train_X = df_train.drop('sales', axis=1)
#     train_y = df_train['sales']
#     test_X = df_test.drop('sales', axis=1)
#     test_y = df_test['sales']

#     model.fit(train_X, train_y)
#     yhat = model.predict(test_X)
#     error = mape(yhat, test_y)
#     print("MAPE Score for %s, is %.5f" %(key, error))

In [ ]:
# import lightgbm as lgbm

# df_dict = {'All': df, 'N': df_N, 'S': df_S, 'E': df_E, 'W': df_W, 'NE': df_NE, 'NW': df_NW, 'SE': df_SE, 'SW': df_SW}
# for key, value in df_dict.items():
#     df = df_dict[key]
    
#     df_test = df[df['date'] >= split_point].copy()
#     df_test.drop('date', axis=1, inplace=True)
#     df_test.reset_index(drop=True, inplace=True)
#     df_train = df[df['date'] < split_point].copy()
#     df_train.drop('date', axis=1, inplace=True)
#     df_train.reset_index(drop=True, inplace=True)

#     model = lgbm.LGBMRegressor(n_estimators=1000)

#     train_X = df_train.drop('sales', axis=1)
#     train_y = df_train['sales']
#     test_X = df_test.drop('sales', axis=1)
#     test_y = df_test['sales']

#     model.fit(train_X, train_y)
#     yhat = model.predict(test_X)
#     error = mape(yhat, test_y)
#     print("MAPE Score for %s, is %.5f" %(key, error))

In [ ]:
# # Predictions for June
# df_train = df[df['date'] >= split_point].copy().drop('date', axis=1)
# df_test = df[df['date'] < split_point].copy().drop('date', axis=1)

# X_train = df_train.drop(['sales previous month'], axis=1)
# X_train.rename(columns={'sales':'sales previous month'}, inplace=True)
# y_train = df_train['sales']
# X_test = df_test.drop(['sales previous month'], axis=1)
# X_test.rename(columns={'sales':'sales previous month'}, inplace=True)
# y_test = df_test['sales']

# model = XGBRegressor(n_estimators=1000)

# model.fit(X_train, y_train)
# yhat = model.predict(X_test)

# Make feature sales next month and set y to be sales next month
df_train = df[df['date'] < split_point].copy().drop('date', axis=1)
df_test = df[df['date'] >= split_point].copy().drop('date', axis=1)

X_train = df_train
y_train = df_train['sales next month']

X_test = df_test
y_test = df_test['sales next month']

model = XGBRegressor(objective='reg:squarederror', n_estimators=1000,seed =10)
model.fit(X_train, y_train)
yhat = model.predict(X_test)

In [ ]:
pred = df_test_copy[['Warehouse id', 'store_id', 'date']].copy()
pred['Next Month Pred'] = yhat
pred['actual'] = y_test.copy().reset_index(drop=True)
pred.head(30)

,Warehouse id,store_id,date,Next Month Pred,actual
0,Wh-2,1,2021-01-01,10.934340,11.0
1,Wh-2,1,2021-02-01,5.026628,5.0
2,Wh-2,1,2021-03-01,5.026628,5.0
3,Wh-2,1,2021-04-01,5.026628,5.0
4,Wh-2,1,2021-05-01,35.447704,36.0
5,Wh-1,1,2021-01-01,13.987426,14.0
6,Wh-1,1,2021-02-01,15.994898,16.0
7,Wh-1,1,2021-03-01,8.010183,8.0
8,Wh-1,1,2021-04-01,12.036793,12.0
9,Wh-1,1,2021-05-01,35.447704,36.0
